In [1]:
%run ../MLHCExperiment-Common-Val17Test18.ipynb

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
all_visits = read_data()
all_visits_weekday = all_visits.where(all_visits["isweekday"] == 0).dropna()
all_visits_weekday = all_visits_weekday.where(all_visits_weekday["AdjCount"] > 0).dropna()
all_visits_weekend = all_visits.where(all_visits["isweekday"] == 1).dropna()
all_visits_weekend = all_visits_weekend.where(all_visits_weekend["AdjCount"] > 0).dropna()

In [3]:
def run_dynamic_baseline(name, features, label):
    print(name)
    min_max_scaler = preprocessing.MinMaxScaler()
    
    X_weekday = all_visits_weekday[features]
    y_weekday = all_visits_weekday[label]

    X_train_weekday, y_train_weekday, _, _, \
    X_test_weekday, y_test_weekday = split_for_baseline_and_nn(X_weekday, y_weekday)

    regr_weekday = linear_model.LinearRegression()
    regr_weekday.fit(min_max_scaler.fit_transform(X_train_weekday), y_train_weekday.values.ravel())
    y_pred_weekday = regr_weekday.predict(min_max_scaler.transform(X_test_weekday))
    
    #print('Weekday Coefficients: \n', regr_weekday.coef_)
    #print("\n Weekday Mean squared error: %.2f" % mean_squared_error(y_test_weekday, y_pred_weekday))
    #print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test_weekday, y_pred_weekday)))
    #print('R2 score: %.2f' % r2_score(y_test_weekday, y_pred_weekday))
    
    X_weekend = all_visits_weekend[features]
    y_weekend = all_visits_weekend[label]

    X_train_weekend, y_train_weekend, _, _, \
    X_test_weekend, y_test_weekend = split_for_baseline_and_nn(X_weekend, y_weekend)

    regr_weekend = linear_model.LinearRegression()
    regr_weekend.fit(min_max_scaler.fit_transform(X_train_weekend), y_train_weekend.values.ravel())
    y_pred_weekend = regr_weekend.predict(min_max_scaler.transform(X_test_weekend))

    #print('Weekend Coefficients: \n', regr_weekend.coef_)
    #print("\n Weekend Mean squared error: %.2f" % mean_squared_error(y_test_weekend, y_pred_weekend))
    #print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test_weekend, y_pred_weekend)))
    #print('R2 score: %.2f' % r2_score(y_test_weekend, y_pred_weekend))

    #plot_weekday_weekend_outputs("Linear Regression", X_train_weekday, X_test_weekday, y_train_weekday,
    #                             y_pred_weekday, y_test_weekday, X_train_weekend, X_test_weekend,
    #                             y_train_weekend, y_pred_weekend, y_test_weekend)
    
    overall_mse = mse_weighted_avg(y_test_weekday, y_pred_weekday, y_test_weekend, y_pred_weekend)
    print("\n Overall Mean squared error: %.3f" % overall_mse)
    print("Root mean squared error: %.3f\n\n" % np.sqrt(overall_mse))
    
    pred_values = X_test_weekday
    pred_values["pred"] = y_pred_weekday

    pred_values_wknd = X_test_weekend
    pred_values_wknd["pred"] = y_pred_weekend

    pred = pred_values.append(pred_values_wknd)
    pred = pred.sort_index(axis=0)
    return pred

In [4]:
pred_all = run_dynamic_baseline("Dynamic baseline, with no lag, all visits", BASE_FEATURES, ALL_VISITS_LABEL)
pred_r = run_dynamic_baseline("Dynamic baseline, with no lag, resp visits", BASE_FEATURES, ICD8_LABEL)
pred_nr = run_dynamic_baseline("Dynamic baseline, with no lag, non resp visits", BASE_FEATURES, ICD8OPP_LABEL)

Dynamic baseline, with no lag, all visits

 Overall Mean squared error: 1.578
Root mean squared error: 1.256




/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Dynamic baseline, with no lag, resp visits

 Overall Mean squared error: 0.474
Root mean squared error: 0.688


Dynamic baseline, with no lag, non resp visits

 Overall Mean squared error: 0.907
Root mean squared error: 0.952




In [5]:
for val in pred_all["pred"].values:
    print(val)

3.07177734375
8.41796875
8.08544921875
0.482421875
6.400390625
3.8701505582764515
2.053245990484206
10.2978515625
8.8154296875
8.38134765625
8.2294921875
8.705078125
3.7664303918470186
1.9369414744478455
10.82470703125
9.31005859375
8.6494140625
7.87841796875
8.9619140625
3.9116417369197483
2.0944880577812626
11.0224609375
9.21630859375
8.6240234375
8.16845703125
8.71630859375
4.014637175367782
2.2152945663060315
11.07763671875
9.0654296875
8.443359375
8.4111328125
8.70166015625
4.207290081353746
2.3748054139020702
10.81640625
9.26708984375
8.619140625
8.296875
8.91650390625
4.172174090826478
2.2931324419093646
10.79931640625
8.998046875
8.57080078125
8.419921875
8.84912109375
3.963881520485853
1.8169331544492096
10.4521484375
8.7861328125
8.58056640625
8.10302734375
8.4462890625
4.0906635838572845
2.222087280699436
10.958984375
9.23388671875
8.7783203125
8.4521484375
8.423828125
4.089626398594193
2.2766403725853324
10.77880859375
9.09375
4.6513671875
8.73779296875
4.000266836468199
2.

In [6]:
for val in pred_r["pred"].values:
    print(val)

1.39544677734375
2.82391357421875
2.73138427734375
0.3385009765625
2.00396728515625
1.5084126984325164
0.8207044409925097
3.8441162109375
3.04736328125
2.92779541015625
2.930267333984375
2.95880126953125
1.486962794323615
0.8035190247970583
3.984344482421875
3.1876220703125
2.975311279296875
2.779632568359375
3.110107421875
1.4535893554363992
0.7722805328824771
4.03814697265625
3.1458740234375
2.991485595703125
2.956146240234375
3.07122802734375
1.6394581364449827
0.9499760748947859
4.04345703125
3.117919921875
2.96319580078125
3.025421142578125
3.028778076171875
1.7096490098426034
1.0049219457292426
4.13909912109375
3.375152587890625
3.167938232421875
3.1356201171875
3.2945556640625
1.6042394213306022
0.8711769824001396
4.01763916015625
3.21258544921875
3.08831787109375
3.074920654296875
3.124786376953125
1.4913935191901797
0.6298528428458852
3.769287109375
2.9044189453125
2.838348388671875
2.7762451171875
2.8472900390625
1.5836114470709792
0.8563098901777972
3.9344482421875
3.1044006

In [7]:
for val in pred_nr["pred"].values:
    print(val)

1.67724609375
5.59423828125
5.35546875
0.1435546875
4.39794921875
2.3617378599131373
1.2325415496385699
6.455078125
5.76904296875
5.455078125
5.2998046875
5.748046875
2.2794675975035954
1.1334224497459573
6.841796875
6.123046875
5.67578125
5.09912109375
5.853515625
2.4580523815711772
1.3222075250275034
6.986328125
6.07177734375
5.634765625
5.21240234375
5.64697265625
2.375179038983592
1.265318491531674
7.03515625
5.9482421875
5.48095703125
5.3857421875
5.673828125
2.4976410713774952
1.3698834680820207
6.67822265625
5.8935546875
5.45263671875
5.1611328125
5.62353515625
2.567934669638082
1.4219554596325286
6.78173828125
5.78564453125
5.48291015625
5.34423828125
5.724609375
2.472488001409261
1.1870803117141062
6.68359375
5.8828125
5.74365234375
5.326171875
5.60009765625
2.5070521367779905
1.36577739053606
7.02587890625
6.130859375
5.80859375
5.53369140625
5.60888671875
2.561464407146786
1.4110201066834822
6.76806640625
5.91259765625
3.24462890625
5.640625
2.482868649046446
1.3472830581313